In [104]:
import argparse
import os
import random

import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm, trange
from transformers import BertJapaneseTokenizer

from eval import eval_vae
from trainer import VAETrainer
from utils import batch_to_device, get_harv_data_loader, get_squad_data_loader
from models import DiscreteVAE, return_mask_lengths

In [2]:
import collections
import json

from transformers import BertJapaneseTokenizer
from tqdm.notebook import tqdm

from qgevalcap.eval import eval_qg
from squad_utils import evaluate, write_predictions
from eval import Result, to_string

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--seed", default=1004, type=int)
parser.add_argument('--debug', dest='debug', action='store_true')
parser.add_argument('--train_dir', default='../data/kosodate/train.json')
parser.add_argument('--dev_dir', default='../data/kosodate/test.json')

parser.add_argument("--max_c_len", default=384, type=int, help="max context length")
parser.add_argument("--max_q_len", default=64, type=int, help="max query length")

parser.add_argument("--model_dir", default="../save/vae-checkpoint-jp", type=str)
parser.add_argument("--epochs", default=30, type=int)
parser.add_argument("--lr", default=1e-3, type=float, help="lr")
parser.add_argument("--batch_size", default=32, type=int, help="batch_size")
parser.add_argument("--weight_decay", default=0.0, type=float, help="weight decay")
parser.add_argument("--clip", default=5.0, type=float, help="max grad norm")

parser.add_argument("--bert_model", default='cl-tohoku/bert-base-japanese-whole-word-masking', type=str)
parser.add_argument('--enc_nhidden', type=int, default=300)
parser.add_argument('--enc_nlayers', type=int, default=1)
parser.add_argument('--enc_dropout', type=float, default=0.2)
parser.add_argument('--dec_a_nhidden', type=int, default=300)
parser.add_argument('--dec_a_nlayers', type=int, default=1)
parser.add_argument('--dec_a_dropout', type=float, default=0.2)
parser.add_argument('--dec_q_nhidden', type=int, default=900)
parser.add_argument('--dec_q_nlayers', type=int, default=2)
parser.add_argument('--dec_q_dropout', type=float, default=0.3)
parser.add_argument('--nzqdim', type=int, default=50)
parser.add_argument('--nza', type=int, default=20)
parser.add_argument('--nzadim', type=int, default=10)
parser.add_argument('--lambda_kl', type=float, default=0.1)
parser.add_argument('--lambda_info', type=float, default=1.0)

args = parser.parse_args([])

In [4]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [5]:
if args.debug:
    args.model_dir = "./dummy"
# set model dir
model_dir = args.model_dir
os.makedirs(model_dir, exist_ok=True)
args.model_dir = os.path.abspath(model_dir)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [6]:
tokenizer = BertJapaneseTokenizer.from_pretrained(args.bert_model)
train_data = get_squad_data_loader(tokenizer, args.train_dir,
                                     shuffle=False, args=args)
eval_data = get_squad_data_loader(tokenizer, args.dev_dir,
                                  shuffle=False, args=args)

100%|█████████████████████████████████████████████████████████████████████████████████| 67/67 [00:00<00:00, 270.14it/s]


In [7]:
args.device = torch.cuda.current_device()
args.device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(args.device)

cuda:1


In [8]:
trainer = VAETrainer(args)

In [9]:
vae = DiscreteVAE(args).to(args.device)

In [10]:
vae.load_state_dict(torch.load(os.path.join(args.model_dir, "best_f1_model.pt"))['state_dict'])

<All keys matched successfully>

In [11]:
trainer.vae = vae

In [12]:
RawResult = collections.namedtuple("RawResult", ["unique_id", "start_logits", "end_logits"])

## evalデータで検証

In [13]:
eval_loader, eval_examples, eval_features = eval_data

In [14]:
all_results = []
qa_results = []
qg_results = {}
res_dict = {}
example_index = -1
qa_posterior_pair = []
qa_prior_pair = []

In [15]:
for batch in tqdm(eval_loader, desc="Eval iter", leave=False, position=4):
    c_ids, q_ids, a_ids, start, end = batch_to_device(batch, args.device)
    batch_size = c_ids.size(0)
    batch_c_ids = c_ids.cpu().tolist()
    batch_q_ids = q_ids.cpu().tolist()
    batch_start = start.cpu().tolist()
    batch_end = end.cpu().tolist()

    batch_posterior_q_ids, \
    batch_posterior_start, batch_posterior_end, \
    posterior_z_prob = trainer.generate_posterior(c_ids, q_ids, a_ids)

    batch_start_logits, batch_end_logits \
    = trainer.generate_answer_logits(c_ids, q_ids, a_ids)

    batch_posterior_q_ids, \
    batch_posterior_start, batch_posterior_end = \
    batch_posterior_q_ids.cpu().tolist(), \
    batch_posterior_start.cpu().tolist(), batch_posterior_end.cpu().tolist()
    posterior_z_prob = posterior_z_prob.cpu()

    batch_prior_q_ids, \
    batch_prior_start, batch_prior_end, \
    prior_z_prob = trainer.generate_prior(c_ids)

    batch_prior_q_ids, \
    batch_prior_start, batch_prior_end = \
    batch_prior_q_ids.cpu().tolist(), \
    batch_prior_start.cpu().tolist(), batch_prior_end.cpu().tolist()
    prior_z_prob = prior_z_prob.cpu()
    
    for i in range(batch_size):
        example_index += 1
        start_logits = batch_start_logits[i].detach().cpu().tolist()
        end_logits = batch_end_logits[i].detach().cpu().tolist()
        eval_feature = eval_features[example_index]
        unique_id = int(eval_feature.unique_id)

        context = to_string(batch_c_ids[i], tokenizer)

        real_question = to_string(batch_q_ids[i], tokenizer)
        posterior_question = to_string(batch_posterior_q_ids[i], tokenizer)
        prior_question = to_string(batch_prior_q_ids[i], tokenizer)

        real_answer = to_string(batch_c_ids[i][batch_start[i]:(batch_end[i] + 1)], tokenizer)
        posterior_answer = to_string(batch_c_ids[i][batch_posterior_start[i]:(batch_posterior_end[i] + 1)], tokenizer)
        prior_answer = to_string(batch_c_ids[i][batch_prior_start[i]:(batch_prior_end[i] + 1)], tokenizer)

        all_results.append(Result(context=context,
                                  real_question=real_question,
                                  posterior_question=posterior_question,
                                  prior_question=prior_question,
                                  real_answer=real_answer,
                                  posterior_answer=posterior_answer,
                                  prior_answer=prior_answer,
                                  posterior_z_prob=posterior_z_prob[i],
                                  prior_z_prob=prior_z_prob[i]))
        
        qa_prior_pair.append([prior_question, prior_answer])
        qa_posterior_pair.append([posterior_question, posterior_answer])

        qg_results[unique_id] = posterior_question
        res_dict[unique_id] = real_question
        qa_results.append(RawResult(unique_id=unique_id,
                                    start_logits=start_logits,
                                    end_logits=end_logits))

In [16]:
output_prediction_file = os.path.join(args.model_dir, "pred.json")
write_predictions(eval_examples, eval_features, qa_results, n_best_size=20,
                  max_answer_length=100, do_lower_case=True, 
                  output_prediction_file=output_prediction_file,
                  verbose_logging=False,
                  version_2_with_negative=False,
                  null_score_diff_threshold=0,
                  noq_position=True)

In [17]:
with open(args.dev_dir, "r", encoding='utf-8') as f:
    dataset_json = json.load(f)
    dataset = dataset_json["data"]
with open(os.path.join(args.model_dir, "pred.json")) as prediction_file:
    predictions = json.load(prediction_file)

In [18]:
predictions['000']

'窓口 で 妊娠 届 を ご 記入 いただ ##き 、 母子 手帳 を お 渡し し ます 。 住民 票 の 世帯 が 別 の 方 が 代理 で 窓口 に 来 ##ら れる 場合 は 、 委任 状 が 必要 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR ##L )'

### prior: (C) -> (C, Zq, Za) -> (Q, A)

In [19]:
for pair in qa_prior_pair: 
    print(pair[0])
    print(pair[1], '\n')

母子 手帳 の 他 に 窓口 し まし た が 、 母子 手帳 の 他 に 窓口 に 必要 です か ?
窓口 で 妊娠 届 を ご 記入 い た だ き 、 母子 手帳 を お 渡し し ます 。 住民 票 の 世帯 が 別 の 方 が 代理 で 窓口 に 来 ら れる 場合 は 、 委任 状 が 必要 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

● ● 県内 で 引っ越し たら 、 子ども の 受 診 票 は ?
1 歳 6 か月 健 診 の 受 診 券 は 、 ○ ○ 市内 のみ で の ご 利用 に なり ます 。 それ 以外 の 受 診 票 は 、 ● ● 県内 共通 に なり ます ので 、 そのまま お 使い い た だ け ます 。 各 自治体 独自 の サービス が 受け られる 場合 が ある ので 、 引っ越し 先 の 自治体 に お 問い合わせ ください 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

9 ・ 7 か月 健 診 の 券 を 失 くし て しまい まし た 。
受 診 票 を 再 発行 し ます 。 ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) へ お 問い合わせ ください 。 [UNK] お 問い合わせ ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) ( 電話 番号 )/( 開 庁 時間 ) 

予 診 が やっ て き ませ ん 。
○ ○ 市 で は 市内 に 住民 登録 の ある お 子 様 へ 予防 接種 す べき 時期 が 近づい たら 予防 接種 記録 票 を お 送り し て い ます 。 ただし 、 転入 さ れる 前 に 発送 時期 を 過ぎ て しまっ た 分 について は 自動 発送 は さ れ ませ ん 。 この 場合 、 予防 接種 記録 票 の 中 で 未 接種 分 について は 、 接種 歴 を もと に 不足 分 を お 送り し ます ので 、 母子 手帳 等 を ご 用意 の 上 、 お 電話 または 窓口 で お 問い合わせ ください 。 [UNK] お 問い合わせ ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) ( 電話 

### posterior: (C, Q, A) -> (C, Zq, Za) -> (Q, A)

In [20]:
for pair in qa_posterior_pair:
    print(pair[0])
    print(pair[1], '\n')

母子 手帳 の 他 に 窓口 し まし た が 、 母子 手帳 の 他 に 窓口 に 必要 です か ?
窓口 で 妊娠 届 を ご 記入 い た だ き 、 母子 手帳 を お 渡し し ます 。 住民 票 の 世帯 が 別 の 方 が 代理 で 窓口 に 来 ら れる 場合 は 、 委任 状 が 必要 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

● ● 県 外 へ 引っ越し たら 、 子ども の 受 診 票 は ?
受 診 票 は ご 利用 い た だ け なく なり ます 。 引っ越し 先 の 自治体 に お 問い合わせ ください 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

7 ・ 7 か月 健 診 の 券 を 失 くし て しまい まし た 。
受 診 票 を 再 発行 し ます 。 ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) へ お 問い合わせ ください 。 [UNK] お 問い合わせ ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) ( 電話 番号 )/( 開 庁 時間 ) 

予 診 が やっ て き ませ ん 。
○ ○ 市 で は 市内 に 住民 登録 の ある お 子 様 へ 予防 接種 す べき 時期 が 近づい たら 予防 接種 記録 票 を お 送り し て い ます 。 ただし 、 転入 さ れる 前 に 発送 時期 を 過ぎ て しまっ た 分 について は 自動 発送 は さ れ ませ ん 。 この 場合 、 予防 接種 記録 票 の 中 で 未 接種 分 について は 、 接種 歴 を もと に 不足 分 を お 送り し ます ので 、 母子 手帳 等 を ご 用意 の 上 、 お 電話 または 窓口 で お 問い合わせ ください 。 [UNK] お 問い合わせ ( 自治体 の 担当 課 や 子育て センター 等 の 名称 ) ( 電話 番号 )/( 開 庁 時間 ) 

療 育 医療 の 申請 手続 を 教え て ください 。
保護 者 の 住所 地 の 保健 所 に 申請 し ます 。 感染 症 法 の 医療 費 助成 申請 と 同時に おこ ない ます 。 1 療 育 医療

In [21]:
ret = evaluate(dataset, predictions)
bleu = eval_qg(res_dict, qg_results)

In [22]:
ret

{'exact_match': 73.13432835820896, 'f1': 85.23131281829728}

In [23]:
bleu*100

34.229906335444866

## trainデータで検証

In [24]:
train_loader, train_examples, train_features = train_data

In [25]:
all_results = []
qa_results = []
qg_results = {}
res_dict = {}
example_index = -1
qa_posterior_pair = []
qa_prior_pair = []

In [26]:
for batch in tqdm(train_loader, desc="Eval iter", leave=False, position=4):
    c_ids, q_ids, a_ids, start, end = batch_to_device(batch, args.device)
    batch_size = c_ids.size(0)
    batch_c_ids = c_ids.cpu().tolist()
    batch_q_ids = q_ids.cpu().tolist()
    batch_start = start.cpu().tolist()
    batch_end = end.cpu().tolist()

    batch_posterior_q_ids, \
    batch_posterior_start, batch_posterior_end, \
    posterior_z_prob = trainer.generate_posterior(c_ids, q_ids, a_ids)

    batch_start_logits, batch_end_logits \
    = trainer.generate_answer_logits(c_ids, q_ids, a_ids)

    batch_posterior_q_ids, \
    batch_posterior_start, batch_posterior_end = \
    batch_posterior_q_ids.cpu().tolist(), \
    batch_posterior_start.cpu().tolist(), batch_posterior_end.cpu().tolist()
    posterior_z_prob = posterior_z_prob.cpu()

    batch_prior_q_ids, \
    batch_prior_start, batch_prior_end, \
    prior_z_prob = trainer.generate_prior(c_ids)

    batch_prior_q_ids, \
    batch_prior_start, batch_prior_end = \
    batch_prior_q_ids.cpu().tolist(), \
    batch_prior_start.cpu().tolist(), batch_prior_end.cpu().tolist()
    prior_z_prob = prior_z_prob.cpu()
    
    for i in range(batch_size):
        example_index += 1
        start_logits = batch_start_logits[i].detach().cpu().tolist()
        end_logits = batch_end_logits[i].detach().cpu().tolist()
        train_feature = train_features[example_index]
        unique_id = int(train_feature.unique_id)

        context = to_string(batch_c_ids[i], tokenizer)

        real_question = to_string(batch_q_ids[i], tokenizer)
        posterior_question = to_string(batch_posterior_q_ids[i], tokenizer)
        prior_question = to_string(batch_prior_q_ids[i], tokenizer)

        real_answer = to_string(batch_c_ids[i][batch_start[i]:(batch_end[i] + 1)], tokenizer)
        posterior_answer = to_string(batch_c_ids[i][batch_posterior_start[i]:(batch_posterior_end[i] + 1)], tokenizer)
        prior_answer = to_string(batch_c_ids[i][batch_prior_start[i]:(batch_prior_end[i] + 1)], tokenizer)

        all_results.append(Result(context=context,
                                  real_question=real_question,
                                  posterior_question=posterior_question,
                                  prior_question=prior_question,
                                  real_answer=real_answer,
                                  posterior_answer=posterior_answer,
                                  prior_answer=prior_answer,
                                  posterior_z_prob=posterior_z_prob[i],
                                  prior_z_prob=prior_z_prob[i]))
        
        qa_prior_pair.append([prior_question, prior_answer])
        qa_posterior_pair.append([posterior_question, posterior_answer])

        qg_results[unique_id] = posterior_question
        res_dict[unique_id] = real_question
        qa_results.append(RawResult(unique_id=unique_id,
                                    start_logits=start_logits,
                                    end_logits=end_logits))

In [27]:
output_prediction_file = os.path.join(args.model_dir, "pred.json")
write_predictions(train_examples, train_features, qa_results, n_best_size=20,
                  max_answer_length=100, do_lower_case=True, 
                  output_prediction_file=output_prediction_file,
                  verbose_logging=False,
                  version_2_with_negative=False,
                  null_score_diff_threshold=0,
                  noq_position=True)

In [28]:
with open(args.train_dir, "r", encoding='utf-8') as f:
    dataset_json = json.load(f)
    dataset = dataset_json["data"]
with open(os.path.join(args.model_dir, "pred.json")) as prediction_file:
    predictions = json.load(prediction_file)

In [29]:
for pair in qa_prior_pair:
    print(pair[0])
    print(pair[1], '\n')

母子 手帳 の 受け取り 場所 は どこ です か ?
母子 手帳 は 、 ○ ○ 市役所 本庁 舎 △ △ 階 × × 課 窓口 、 [UNK] [UNK] 出張所 、 .........( その他 の 受け取り 場所 を 適 宜 記載 )......... で 受け 取 れ ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

母子 手帳 は すぐ に 発行 し て もらえ ます か ?
母子 手帳 は 、 妊娠 届 の 内容 を 確認 さ せ て い た だ き 、 その 場 で お 渡し し ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

● ● 県内 で 引っ越し たら 、 妊 婦 健 診 の 受 診 票 は ?
妊 婦 健 診 受 診 票 を お 渡し し て から の 助成 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) < 県内 > 受 診 票 の 妊 婦 歯科 健 診 は 、 ○ ○ 市内 のみ で の ご 利用 に なり ます 。 それ 以外 の 受 診 票 は 、 ● ● 県内 共通 に なり ます ので 、 そのまま お 使い い た だ け ます 。 各 自治体 独自 の サービス が 受け られる 場合 が ある ので 、 引っ越し 先 の 自治体 に お 問い合わせ ください 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

○ ○ 市外 で も 6 ・ 7 か月 健 診 の 受 診 票 は 使え ます か ?
受 診 票 は ○ ○ 市外 でも ● ● 県内 の 契約 医療 機関 で あれ ば お 使い い た だ け ます 。 受 診 希望 の 病院 に お 問い合わせ ください 。 契約 医療 機関 について は こちら を ご [UNK] ください 。 ( 自治体 HP 内 関連 ページ の UR L ) 

● ● 県内 で 引っ越し たら 、 妊 婦 健 診 の 受 診 票 は ?
妊 婦 健 診 受 診 票 を お 渡し し て から の 助成 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ペ

軽自動車 税 の 納税 証明 書 を 取り たい 。
軽自動車 税 の 納税 証明 書 は 、 ( 申請 場所 ( 自治体 の 担当 課 、 支所 ・ 出張所 等 )) で 発行 し て い ます 。 申請 方法 等 、 詳しく は こちら を ご [UNK] ください 。 ( 自治体 HP 内 関連 ページ の UR L ) 

戸籍 [UNK] の 請求 方法 を 教え て ください 。
○ ○ 市 に 本 籍 が ある 人 は 、 窓口 で 交付 を 受ける こと が でき ます 。 必要 な もの 等 、 詳しく は こちら を ご [UNK] ください 。 ( 自治体 HP 内 関連 ページ の UR L ) 



In [30]:
for pair in qa_posterior_pair:
    print(pair[0])
    print(pair[1], '\n')

母子 手帳 の 受け取り 場所 は どこ です か ?
母子 手帳 は 、 ○ ○ 市役所 本庁 舎 △ △ 階 × × 課 窓口 、 [UNK] [UNK] 出張所 、 .........( その他 の 受け取り 場所 を 適 宜 記載 )......... で 受け 取 れ ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

母子 手帳 は すぐ に 発行 し て もらえ ます か ?
母子 手帳 は 、 妊娠 届 の 内容 を 確認 さ せ て い た だ き 、 その 場 で お 渡し し ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

● ● 県内 で 引っ越し たら 、 妊 婦 健 診 の 受 診 票 は ?
妊 婦 健 診 の 受 診 票 は 、 受 診 票 を 受け取っ た 日 より 後 で 、 病院 が 妊 婦 健 診 と 規定 し た 日 に 利用 でき ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

妊 婦 健 診 受 診 票 は ○ ○ 市外 で 使え ます か ?
妊 婦 健 診 の 受 診 票 は 、 ● ● 県内 の 契約 医療 機関 で お 使い い た だ け ます 。 受 診 希望 の 病院 に お 問い合わせ ください 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

妊 婦 健 診 受 診 票 の 余り を 換 金 し て もらえ ます か ?
妊 婦 健 診 に対する 助成 制度 で あり 、 金 券 で は あり ませ ん 。 その ため 換 金 等 は でき ませ ん 。 

妊 婦 健 診 受 診 票 受 取 前 に 病院 に かかっ た 分 は 、 還 付 さ れ ます か ?
妊 婦 健 診 受 診 票 を お 渡し し て から の 助成 に なり ます 。 [UNK] 詳しく は こちら ( 自治体 HP 内 関連 ページ の UR L ) 

妊 婦 健 診 が 14 回 以上 かかり まし た が 、 追加 で 助成 し て もらえ ます か ?
妊 婦 健 診 費用 の 助成 は 14 回 まで に なり ます 

In [31]:
ret = evaluate(dataset, predictions)
bleu = eval_qg(res_dict, qg_results)

Unanswered question 139 will receive score 0.
Unanswered question 177 will receive score 0.


In [32]:
ret

{'exact_match': 71.26050420168067, 'f1': 86.27622391107421}

In [33]:
bleu*100

91.56965195959954

## テキストファイルからドキュメントを作る

In [38]:
all_text = open('../data/kosodate/document_v2.txt', 'r', encoding='utf-8').read()

In [43]:
# 大区分
text_group = all_text.split('\n\n')

In [44]:
len(text_group)

174

In [51]:
for i in range(10):
    print(text_group[i])
    print('----------')


赤ちゃんの成長や性格、生活環境や事情によって解決策が違います。保健師にご相談ください。お話を伺って一緒に考えましょう。
AAA保健相談所：（電話番号）（○時○分から○時○分）
BBB保健相談所：（電話番号）（○時○分から○時○分）
----------
（自治体の発行する広報誌）は、毎月○回（○日・○日）発行し、市内全世帯に各戸配布しています。
発行日を過ぎても届かない場合は、お手数ですが、（自治体の担当課等）へ連絡してください。
◆お問い合わせ
（自治体の担当課等の名称）
（電話番号）／（開庁時間）
▼その他の配布場所等、詳しくはこちら
（自治体HP内関連ページのURL）
----------
<10か月>
受診票を再発行します。
（自治体の担当課や子育てセンター等の名称）へお問い合わせください。
◆お問い合わせ
（自治体の担当課や子育てセンター等の名称）
（電話番号）／（開庁時間）
----------
<1歳>
	<6か月>
	1歳6か月児健診は母子保健法に定められた子どもの定期健診の一つです。子どもの発達の状況を確認するとともに、歯の健診も行います。対象の方には1歳○か月に達する月の（上旬／中旬／下旬）に通知されます。
	▼詳しくはこちら
	（自治体HP内関連ページのURL）
	<保健所>
	1歳児健診は保健所では実施していません。
	○○市では、3・4か月健診、1歳6か月歯科健診、3歳児健診を無料で行っています。
	また、6・7か月健診、9・10か月健診、1歳6か月健診については、受診票を郵送しています。
	●●県内（1歳6か月健診は○○市内のみ）の契約医療機関に受診票をお持ちください。
	契約医療機関についてはこちらをご覧ください。
	（自治体HP内関連ページのURL）
----------
<3歳>
3歳児健診で尿が取れない場合でも、健診後、4歳になる前日までに健診受付時間内に、尿をお持ちいただければ追加で検査できます。
お子さんの成長に合わせてお持ちください。
また、健診の朝の尿が難しい場合は、健診会場でお取りいただくこともできます。
3歳児健診は子どもの3歳の誕生月に通知されますので、指定の保健相談所にて受診してください。
▼詳しくはこちら
（自治体HP内関連ページのURL）
健診の日時や場所はお住まいの地域により異なります。
次の健診の日時等は

In [52]:
text_group_tokens = [tokenizer.tokenize(t) for t in text_group]

In [53]:
for i in range(10):
    print(text_group_tokens[i])
    print('----------')

['赤ちゃん', 'の', '成長', 'や', '性格', '、', '生活', '環境', 'や', '事情', 'によって', '解決', '策', 'が', '違い', 'ます', '。', '保健', '師', 'に', 'ご', '相談', 'ください', '。', 'お', '##話', 'を', '伺', '##っ', 'て', '一緒', 'に', '考え', 'ましょ', 'う', '。', 'AAA', '保健', '相談', '所', ':', '##(', '電話', '番号', ')(', '##○', '時', '○', '分', 'から', '○', '時', '○', '分', ')', 'BB', '##B', '保健', '相談', '所', ':', '##(', '電話', '番号', ')(', '##○', '時', '○', '分', 'から', '○', '時', '○', '分', ')']
----------
['(', '自治体', 'の', '発行', 'する', '広報', '誌', ')', 'は', '、', '毎月', '○', '回', '(', '##○', '日', '・', '○', '日', ')', '発行', 'し', '、', '市内', '全', '世帯', 'に', '各', '##戸', '配布', 'し', 'て', 'い', 'ます', '。', '発行', '日', 'を', '過ぎ', 'て', 'も', '届か', 'ない', '場合', 'は', '、', 'お', '##手', '##数', 'です', 'が', '、', '(', '自治体', 'の', '担当', '課', '等', ')', 'へ', '連絡', 'し', 'て', 'ください', '。', '[UNK]', 'お', '問い合わせ', '(', '自治体', 'の', '担当', '課', '等', 'の', '名称', ')', '(', '電話', '番号', ')/', '##(', '開', '庁', '時間', ')', '[UNK]', 'その他', 'の', '配布', '場所', '等', '、', '詳しく', 'は', 'こちら', '(', '自治体', 'HP', 

In [55]:
# context_length <= 256 となるように分割する
max([len(t) for t in text_group_tokens])

5375

In [67]:
documents = []
for t in text_group_tokens:
    if len(t) <= 256:
        documents.append(t)
    else:
        tmp = t[:]
        while len(tmp) > 256:
            documents.append(tmp[:256])
            # 半分ずらす
            tmp = tmp[128:]
        if len(tmp) > 0:
            documents.append(tmp)

In [68]:
len(documents)

412

In [69]:
max([len(t) for t in documents])

256

In [70]:
print(documents[:100])

[['赤ちゃん', 'の', '成長', 'や', '性格', '、', '生活', '環境', 'や', '事情', 'によって', '解決', '策', 'が', '違い', 'ます', '。', '保健', '師', 'に', 'ご', '相談', 'ください', '。', 'お', '##話', 'を', '伺', '##っ', 'て', '一緒', 'に', '考え', 'ましょ', 'う', '。', 'AAA', '保健', '相談', '所', ':', '##(', '電話', '番号', ')(', '##○', '時', '○', '分', 'から', '○', '時', '○', '分', ')', 'BB', '##B', '保健', '相談', '所', ':', '##(', '電話', '番号', ')(', '##○', '時', '○', '分', 'から', '○', '時', '○', '分', ')'], ['(', '自治体', 'の', '発行', 'する', '広報', '誌', ')', 'は', '、', '毎月', '○', '回', '(', '##○', '日', '・', '○', '日', ')', '発行', 'し', '、', '市内', '全', '世帯', 'に', '各', '##戸', '配布', 'し', 'て', 'い', 'ます', '。', '発行', '日', 'を', '過ぎ', 'て', 'も', '届か', 'ない', '場合', 'は', '、', 'お', '##手', '##数', 'です', 'が', '、', '(', '自治体', 'の', '担当', '課', '等', ')', 'へ', '連絡', 'し', 'て', 'ください', '。', '[UNK]', 'お', '問い合わせ', '(', '自治体', 'の', '担当', '課', '等', 'の', '名称', ')', '(', '電話', '番号', ')/', '##(', '開', '庁', '時間', ')', '[UNK]', 'その他', 'の', '配布', '場所', '等', '、', '詳しく', 'は', 'こちら', '(', '自治体', 'HP', '内', '関連'

## 指定のファイルからQA生成

In [93]:
def document_preprocess(documents, device):
    res = []
    for d in documents:
        res.append(['[CLS]'])
        res[-1].extend(d)
        res[-1].append('[SEP]')
        while len(res[-1]) < 256+2:
            res[-1].append('[PAD]')
        res[-1] =  tokenizer.convert_tokens_to_ids(res[-1])
    return torch.tensor(res, dtype=torch.long, device=device)

In [139]:
def docs_to_qas(c_ids, trainer, tokenizer):
    
    res = []
    
    tmp = [[] for i in range(3)]
    
    for n in range(3):
        batch_prior_q_ids, batch_prior_start, batch_prior_end, prior_z_prob = trainer.generate_prior(c_ids)
    
        for i in range(len(c_ids)):
            dic = {}
            dic['context'] = to_string(c_ids[i], tokenizer)
            dic['question'] = to_string(batch_prior_q_ids[i], tokenizer)
            dic['answer'] = to_string(c_ids[i][batch_prior_start[i]:(batch_prior_end[i] + 1)], tokenizer)
            tmp[n].append(dic)
    
    for i in range(len(c_ids)):
        for n in range(3):
            res.append(tmp[n][i])
    
    return res

In [140]:
def qa_generation(text_dir, save_dir, trainer, tokenizer, save_encoding='utf-8'):
    all_text = open(text_dir, 'r', encoding='utf-8').read()
    
    # 大区分
    text_group = all_text.split('\n\n')
    text_group_tokens = [tokenizer.tokenize(t) for t in text_group]
    
    # context_length <= 256 となるように分割する
    documents = []
    for t in text_group_tokens:
        if len(t) <= 256:
            documents.append(t)
        else:
            tmp = t[:]
            while len(tmp) > 256:
                documents.append(tmp[:256])
                # 半分ずらす
                tmp = tmp[128:]
            if len(tmp) > 0:
                documents.append(tmp)
                
    # ドキュメントの前処理
    document_ids = document_preprocess(documents, args.device)
    
    batch = []
    idx = 0
    while idx < len(document_ids):
        batch.append(document_ids[idx:idx+32])
        idx += 32
    
    results = []
    for data in batch:
        results.extend( docs_to_qas(data, trainer, tokenizer) )

    df = pd.DataFrame(results, columns=['question', 'answer'])
    df.to_csv(save_dir, encoding=save_encoding)
    
    return df

In [141]:
text_dir= '../data/kosodate/document_v2.txt'
save_dir = '../data/kosodate/result.csv'

In [142]:
qa_generation(text_dir, save_dir, trainer, tokenizer, save_encoding='cp932')

,question,answer
0,出産 の 成長 や 出産 の 相談 は どこ に すれ ば いい です か ?,赤ちゃん の 成長 や 性格 、 生活 環境 や 事情 によって 解決 策 が 違い ます ...
1,出産 の 成長 や 出産 の 相談 は どこ に すれ ば いい です か ?,BBB 保健 相談 所 :( 電話 番号 )(○ 時 ○ 分 から ○ 時 ○ 分 )
2,出産 の 成長 や 出産 の 相談 は どこ に すれ ば いい です か ?,赤ちゃん の 成長 や 性格 、 生活 環境 や 事情 によって 解決 策 が 違い ます ...
3,( 自治体 の 古 着 など の 配布 場所 を 教え て ください 。,[UNK] その他 の 配布 場所 等 、 詳しく は こちら ( 自治体 HP 内 関連 ...
4,( 自治体 の 古 着 など の 配布 場所 を 教え て ください 。,[UNK] その他 の 配布 場所 等 、 詳しく は こちら ( 自治体 HP 内 関連 ...
...,...,...
1231,養子 縁組 の 届出 場所 を 教え て ください 。,養子 縁組 は 届出 によって 法的 に 嫡出 親子 関係 が 成立 し ます 。 養子 縁...
1232,養子 縁組 の 届出 場所 を 教え て ください 。,養子 縁組 は 届出 によって 法的 に 嫡出 親子 関係 が 成立 し ます 。 養子 縁...
1233,○ ○ 市 の 駐輪 場 について 教え て ください 。,○ ○ 市 の 駐輪場 について は こちら を ご覧 ください 。 ( 自治体 HP 内 ...
1234,○ ○ 市 の 駐輪 場 について 教え て ください 。,○ ○ 市 の 駐輪場 について は こちら を ご覧 ください 。 ( 自治体 HP 内 ...
